# Transfer Characteristics of CMOS inverter

In [1]:
from ipywidgets import interact, interactive, IntSlider, Layout
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from math import pi

# Variable declaration

In [2]:
vd = np.linspace(0,2.1,100)  # Drain voltage
vtn  = 0.4                   # threshold voltage for NMOS
vtp  = -0.4                 # threshold voltage for PMOS
betan = 50e-6               # beta for NMOS
betap = 20e-6               # beta for PMOS
Ids = np.zeros(100) #for nmos
Idp= np.zeros(100) #for pmos
lambda_n = 0.1              # Channel modulation effect
lambda_p = 0.1              # Channel modulation effect

# Function definitions

In [3]:
def current(vgs,vds,Betan,Betap):
    vv = 0
    idd= 0
    for i,v in enumerate(vds):
        #NMOS
        if vgs <= vtn:   #  cutoff 
            Ids[i] = 0
        elif v <= (vgs-vtn):    # linear region
            Ids[i] = ( Betan * (vgs - vtn - v/2)*v )
        else:   # saturation region
            Ids[i] = ( Betan/2*(vgs-vtn)**2)*(1+v*lambda_n)
        #PMOS
        vsd = 5 - v
        vsg = 5 - vgs
        if vsg <= vtp:  # cutoff
            Idp[i] = 0
        elif vsd <= (vsg-vtp): # linear region
            Idp[i] = ( Betap * (vsg - vtp - vsd/2)*vsd )
        else:   # saturation region
            Idp[i] = ( Betap/2*(vsg-vtp)**2)*(1+vsd*lambda_p)
    return Ids, Idp

def finding_intersection(vgs,vds,Betan,Betap):
    (Ids,Idp) = current(vgs,vds,Betan,Betap)
    Idiff = Ids - Idp
    vv = 0
    idd = 0
    for i in range(1,len(Idp)-1):
        if Idiff[i]*Idiff[i+1] <= 0:
            vv = vds[i]
            idd = Ids[i]
#             print(vv,idd)
            break
    return vv, idd, Ids, Idp

vinput = np.linspace(0,5,100)
vout = np.zeros(100)
Iout = np.zeros(100)
for i,v in enumerate(vinput):
    vv, idd, Ids, Idp = finding_intersection(v,vd,betan,betap)
    vout[i] = vv
    Iout[i] = idd

# Interactive Plot

In [4]:
def drawing_plot(vgs):
    vv, idd, Ids, Idp = finding_intersection(vgs,vd,betan,betap)
#     fig, (ax1, ax2, ax3, ax4) = plt.subplots(2, 2)
    fig=plt.figure(figsize=(9, 6), dpi= 80)
    plt.subplot(221)
    plt.title('I-V characteristics')
    plt.plot(vd,Ids*1000)
    plt.xlabel('Drain Voltage (V)')
    
    plt.ylabel('Drain Current (mA)')
    plt.plot(vd,Idp*1000)
    plt.legend(['NMOS', 'PMOS'])
    plt.plot(vv,idd*1000,'ro')
    plt.axis([0,5,0,0.2])
#     plt.show()
    plt.subplot(222)
    plt.title('Transfer characteristics of CMOS')
    plt.plot(vinput,vout)
    plt.xlabel('Input Voltage (V)')
    plt.ylabel('Output Voltage (V)')
    plt.plot(vgs,vv,'ro')
    
#     plt.show()
    plt.subplot(223)
    plt.title('Current through CMOS circuit')
    plt.plot(vinput,Iout*1000)
    plt.xlabel('Input Voltage (V)')
    plt.ylabel('Drain Current (mA)')
    plt.plot(vgs,idd*1000,'ro')
    
    
    plt.subplot(224)
    plt.title('Power consumption')
    plt.plot(vinput,5*Iout*1000)
    plt.xlabel('Input Voltage (V)')
    plt.ylabel('Power (mW)')
    plt.plot(vgs,5*idd*1000,'ro')
    plt.tight_layout()
    plt.show()
   
interact(drawing_plot,vgs =(0,5,0.1))

interactive(children=(FloatSlider(value=2.0, description='vgs', max=5.0), Output()), _dom_classes=('widget-int…

<function __main__.drawing_plot>

# Variation w.r.t $K_n$ and $K_p$

In [6]:
vpulse = np.zeros(12);
vpulse = np.append(vpulse, np.linspace(0,5,12))
vpulse = np.append(vpulse, 5*np.ones(52))
vpulse = np.append(vpulse, np.linspace(5,0,12))
vpulse = np.append(vpulse, np.zeros(12))
vres = np.zeros(100);
ires = np.zeros(100);
def drawing_beta_variations(Betap):
    flag1 = 1
    flag2 = 1
    v1 = 100
    v2 = 100
    for i,v in enumerate(vinput):
        vv, idd, Ids, Idp = finding_intersection(v,vd,betan,Betap)
        vv1, idd1, Ids, Idp = finding_intersection(vpulse[i],vd,betan,Betap)
        vout[i] = vv
        Iout[i] = idd
        vres[i] = vv1
        ires[i] = idd1
        if vv <= abs(v-vtn) and flag1 == 1:
            v1 = v
            flag1 = 0
        if (5-vv) >= abs(5-v-vtp) and flag2 == 1:
            v2 = v
            flag2 = 0
    fig=plt.figure(figsize=(6, 6), dpi= 80)
    plt.subplot(211)
    plt.title('Transfer characteristics of CMOS')
    plt.plot(vinput,vout,'k')
    plt.axvline(x=vtn)
    plt.axvline(x=5-vtp)
    plt.axvline(x=v1,color='r')
    plt.axvline(x=v2,color='g')
    plt.xlabel('Input Voltage (V)')
    plt.ylabel('Output Voltage (V)')
    
    plt.subplot(212)
    plt.title('Pulse Shape of CMOS')
    plt.plot(vpulse)
    plt.plot(vres)
    plt.plot(ires*30000)
    plt.xlabel('time (s)')
    plt.ylabel('Voltage (V)')
    plt.legend(['Input', 'Output', 'current'])
#     plt.plot(vgs,vv,'ro')
    plt.tight_layout()
    plt.plot()
    
interact(drawing_beta_variations,Betap =(betan/10,4*betan,betan/10))

interactive(children=(FloatSlider(value=0.0001, description='Betap', max=0.0002, min=5e-06, step=5e-06), Outpu…

<function __main__.drawing_beta_variations>